In [27]:
# import os
# import shutil

# # Define source and destination paths
# source_folder = "./data/realsr_text/test"  # Replace with your source folder path
# hr_folder = "./data/TextZoom/test/HR"      # Replace with your HR folder path
# lr_folder = "./data/TextZoom/test/LR"      # Replace with your LR folder path

# # Create HR and LR folders if they don't exist
# os.makedirs(hr_folder, exist_ok=True)
# os.makedirs(lr_folder, exist_ok=True)

# # Create a mapping of original folder names to new sequential names
# folder_mapping = {}
# current_index = 0

# # Iterate over the subfolders in the source folder
# for folder_name in sorted(os.listdir(source_folder)):
#     original_folder_path = os.path.join(source_folder, folder_name)
#     if os.path.isdir(original_folder_path):
#         # Map the original folder name to a zero-padded sequential name
#         new_folder_name = f"{current_index:03d}"  # Sequential numbering
#         folder_mapping[folder_name] = new_folder_name
#         current_index += 1

# # Walk through the source folder and copy files
# for root, dirs, files in os.walk(source_folder):
#     for file in files:
#         # Determine if the file is for HR or LR
#         if file.startswith("1_") or file.startswith("2_"):
#             # Get the parent folder name
#             parent_folder_name = os.path.basename(root)
#             # Map the parent folder name to the new sequential folder name
#             if parent_folder_name in folder_mapping:
#                 new_folder_name = folder_mapping[parent_folder_name]
#                 # Create the destination folder
#                 destination_path = (
#                     os.path.join(hr_folder, new_folder_name)
#                     if file.startswith("1_")
#                     else os.path.join(lr_folder, new_folder_name)
#                 )
#                 os.makedirs(destination_path, exist_ok=True)
#                 # Copy and rename the file to "00000000.jpg"
#                 new_file_name = "00000000.jpg"
#                 shutil.copy(
#                     os.path.join(root, file),
#                     os.path.join(destination_path, new_file_name),
#                 )


In [35]:
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import torchvision.transforms as T

class TextZoomDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform=None):
        self.lr_paths = sorted([os.path.join(lr_dir, f) for f in os.listdir(lr_dir)])
        self.hr_paths = sorted([os.path.join(hr_dir, f) for f in os.listdir(hr_dir)])
        self.transform = transform

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr = Image.open(self.lr_paths[idx]).convert('RGB')
        hr = Image.open(self.hr_paths[idx]).convert('RGB')
        if self.transform:
            lr = self.transform(lr)
            hr = self.transform(hr)
        return lr, hr

# Transforms for TextZoom
transform = T.Compose([
    T.ToTensor(),
    T.Resize((64, 64)),  # Resize LR images to target size
])

# train_dataset = TextZoomDataset('./data/TextZoom/train/LR', './data/TextZoom/train/HR', transform=transform)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [8]:
import cv2
import numpy as np
import mmagic.models
import mmagic.engine
import torch

# from mmengine.runner import load_checkpoint
# from mmengine.config import Config
# from builder import build_model

In [2]:
# def init_model(config, checkpoint=None):
#     """Initialize a model from config file.

#     Args:
#         config (str or :obj:`mmcv.Config`): Config file path or the config
#             object.
#         checkpoint (str, optional): Checkpoint path. If left as None, the model
#             will not load any weights.
#         device (str): Which device the model will deploy. Default: 'cuda:0'.

#     Returns:
#         nn.Module: The constructed model.
#     """

#     if isinstance(config, str):
#         config = mmengine.Config.fromfile(config)
#     elif not isinstance(config, mmcv.Config):
#         raise TypeError('config must be a filename or Config object, '
#                         f'but got {type(config)}')
#     # config.model.pretrained = None
#     config.test_cfg.metrics = None
#     model = build_model(config.model, test_cfg=config.test_cfg)
#     if checkpoint is not None:
#         checkpoint = load_checkpoint(model, checkpoint)

#     model.cfg = config  # save the config in the model for convenience
#     model.eval()

#     return model

In [9]:
checkpoint_path = './data/checkpoints/realbasicvsr_reds.pth'
config_path = './config.py'
# model = init_model(config_path, checkpoint_path)

In [10]:
from mmengine.registry import MODELS

print(MODELS)

                                                 Registry of model                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Names                                           ┃ Objects                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AdvLoss                                         │ <class 'mmagic.models.losses.adv_loss.AdvLoss'>               │
│ AllegroPipeline                                 │ <class 'mmagic.models.archs.AllegroPipeline'>                 │
│ AllegroTransformer3DModel                       │ <class 'mmagic.models.archs.AllegroTransformer3DModel'>       │
│ AltDiffusionImg2ImgPipeline                     │ <class 'mmagic.models.archs.AltDiffusionImg2ImgPipeline'>     │
│ AltDiffusionPipeline                            │ <class 'mmagic.model

In [11]:

from mmengine import Config

In [36]:
config = Config.fromfile(config_path)

config.train_dataloader.dataset = TextZoomDataset('./data/TextZoom/train/LR', './data/TextZoom/train/HR', transform=transform)
print(config)

Config (path: ./config.py): {'exp_name': 'realbasicvsr_c64b20_1x30x8_lr5e-5_150k_reds', 'scale': 4, 'model': {'type': 'RealBasicVSR', 'generator': {'type': 'RealBasicVSRNet', 'mid_channels': 64, 'num_propagation_blocks': 20, 'num_cleaning_blocks': 20, 'dynamic_refine_thres': 255, 'spynet_pretrained': 'https://download.openmmlab.com/mmediting/restorers/basicvsr/spynet_20210409-c6c1bd09.pth', 'is_fix_cleaning': False, 'is_sequential_cleaning': False}, 'discriminator': {'type': 'UNetDiscriminatorWithSpectralNorm', 'in_channels': 3, 'mid_channels': 64, 'skip_connection': True}, 'pixel_loss': {'type': 'L1Loss', 'loss_weight': 1.0, 'reduction': 'mean'}, 'cleaning_loss': {'type': 'L1Loss', 'loss_weight': 1.0, 'reduction': 'mean'}, 'perceptual_loss': {'type': 'PerceptualLoss', 'layer_weights': {'2': 0.1, '7': 0.1, '16': 1.0, '25': 1.0, '34': 1.0}, 'vgg_type': 'vgg19', 'perceptual_weight': 1.0, 'style_weight': 0, 'norm_img': False}, 'gan_loss': {'type': 'GANLoss', 'gan_type': 'vanilla', 'loss_w

In [37]:
from mmengine.runner import Runner

runner = Runner.from_cfg(config)
runner.train()

01/20 04:57:51 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 150972512
    GPU 0: NVIDIA GeForce RTX 4060
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.5.1+cu124
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 192930154
  - Intel(R) oneAPI Math Kernel Library Version 2024.2.2-Product Build 20240823 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.4
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-ge

Exception in thread Exception ignored in: <function _ConnectionBase.__del__ at 0x000001B8DC759A80>
Traceback (most recent call last):
  File "C:\Python312\Lib\multiprocessing\connection.py", line 133, in __del__
QueueFeederThread:
Traceback (most recent call last):
  File "C:\Python312\Lib\multiprocessing\queues.py", line 259, in _feed
    self._close()
  File "C:\Python312\Lib\multiprocessing\connection.py", line 282, in _close
    reader_close()
  File "C:\Python312\Lib\multiprocessing\connection.py", line 178, in close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid
    self._close()
  File "C:\Python312\Lib\multiprocessing\connection.py", line 282, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\Dan\Projects\alpr-v

SyntaxError: Failed to format the config file, please check the syntax of: 
checkpoint_config=dict(
    by_epoch=False,
    interval=5000,
    save_optimizer=True)
custom_hooks=[
    dict(interp_cfg=dict(
            momentum=0.999),
        interval=1,
        module_keys=(
            'generator_ema',
            ),
        type='ExponentialMovingAverageHook'),
    ]
custom_imports=dict(
    allow_failed_imports=False,
    imports=[
        'mmagic',
        ])
data=dict(
    test=dict(
        gt_folder='./data/TextZoom/test/HR',
        lq_folder='./data/TextZoom/test/LR',
        pipeline=[
            dict(filename_tmpl='{}.jpg',
                interval_list=[
                    1,
                    ],
                type='GenerateSegmentIndices'),
            dict(channel_order='rgb',
                io_backend='disk',
                key='lq',
                type='LoadImageFromFileList'),
            dict(keys=[
                    'lq',
                    ],
           